<a href="https://colab.research.google.com/github/Akash-sahay/Computer-Vision-for-underwater-UAV/blob/main/Working_with_video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
from google.colab import files
import numpy as np

In [ ]:
def compensate_color_exponential(image, alpha_R=1, alpha_G=1, alpha_B=1, depth_map=None):
    """
    Compensates for color attenuation in an underwater image.

    Parameters:
    - image: Input underwater image (in BGR format).
    - alpha_R, alpha_G, alpha_B: Attenuation coefficients for Red, Green, and Blue channels respectively.
    - depth_map: Estimated depth map of the image (optional). If None, a uniform depth will be assumed.

    Returns:
    - compensated_image: Color compensated image.
    """
    # Split the image into individual B, G, R channels
    B, G, R = cv2.split(image)

    # Get image dimensions
    height, width = image.shape[:2]

    # If no depth map is provided, assume a uniform depth for simplicity
    if depth_map is None:
       # Define pink and light blue in RGB
        pink = [255, 192, 203]  # RGB values for pink
        light_blue = [173, 216, 230]  # RGB values for light blue

     # Convert RGB to grayscale-like values to represent depth
        pink_depth_value = np.mean(pink) / 255.0  # Normalize to [0, 1] range
        blue_depth_value = np.mean(light_blue) / 255.0  # Normalize to [0, 1] range

    # Create the depth map
        depth_map = np.zeros((height, width))

    # Fill upper half with pink depth value
        depth_map[:height // 2, :] = pink_depth_value

    # Fill lower half with blue depth value
        depth_map[height // 2:, :] = blue_depth_value

    # Apply the compensation formula for each channel
    compensated_R = R * np.exp(-alpha_R * depth_map)
    compensated_G = G * np.exp(-alpha_G * depth_map)
    compensated_B = B * np.exp(-alpha_B * depth_map)

    # Clip the values to the valid range [0, 255] and convert back to uint8
    compensated_R = np.clip(compensated_R, 0, 255).astype(np.uint8)
    compensated_G = np.clip(compensated_G, 0, 255).astype(np.uint8)
    compensated_B = np.clip(compensated_B, 0, 255).astype(np.uint8)

    # Merge the compensated channels back together
    compensated_image = cv2.merge([compensated_B, compensated_G, compensated_R])

    return compensated_image

In [ ]:
def compensate_color_logarithmic(image, alpha_R=0.5, alpha_G=0.1, alpha_B=0.1, depth_map=None):
    """
    Compensates for color attenuation in an underwater image.

    Parameters:
    - image: Input underwater image (in BGR format).
    - alpha_R, alpha_G, alpha_B: Attenuation coefficients for Red, Green, and Blue channels respectively.
    - depth_map: Estimated depth map of the image (optional). If None, a uniform depth will be assumed.

    Returns:
    - compensated_image: Color compensated image.
    """
    # Split the image into individual B, G, R channels
    B, G, R = cv2.split(image)

    # Get image dimensions
    height, width = image.shape[:2]

   # If no depth map is provided, assume a uniform depth for simplicity
    if depth_map is None:
       # Define pink and light blue in RGB
        pink = [255, 192, 203]  # RGB values for pink
        light_blue = [173, 216, 230]  # RGB values for light blue

     # Convert RGB to grayscale-like values to represent depth
        pink_depth_value = np.mean(pink) / 255.0  # Normalize to [0, 1] range
        blue_depth_value = np.mean(light_blue) / 255.0  # Normalize to [0, 1] range

    # Create the depth map
        depth_map = np.zeros((height, width))

    # Fill upper half with pink depth value
        depth_map[:height // 2, :] = pink_depth_value

    # Fill lower half with blue depth value
        depth_map[height // 2:, :] = blue_depth_value


    # Apply the compensation formula for each channel
    compensated_R = R * np.log(1-alpha_R*R* depth_map)
    compensated_G = G * np.log(1+alpha_G*G* depth_map)
    compensated_B = B * np.log(1+alpha_B*B* depth_map)

    # Clip the values to the valid range [0, 255] and convert back to uint8
    compensated_R = np.clip(compensated_R, 0, 255).astype(np.uint8)
    compensated_G = np.clip(compensated_G, 0, 255).astype(np.uint8)
    compensated_B = np.clip(compensated_B, 0, 255).astype(np.uint8)

    # Merge the compensated channels back together
    compensated_image = cv2.merge([compensated_B, compensated_G, compensated_R])

    return compensated_image

In [ ]:
def increase_contrast(image):
    """
    Increases the contrast of an image using Histogram Equalization.

    Parameters:
    - image: Input image (in BGR format).

    Returns:
    - contrast_image: Contrast-enhanced image.
    """
    # Check if the image is in color
    if len(image.shape) == 3 and image.shape[2] == 3:
        # Convert to YCrCb color space
        ycrcb_image = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)

        # Apply histogram equalization on the Y channel (luminance)
        ycrcb_image[:, :, 0] = cv2.equalizeHist(ycrcb_image[:, :, 0])

        # Convert back to BGR color space
        contrast_image = cv2.cvtColor(ycrcb_image, cv2.COLOR_YCrCb2BGR)
    else:
        # If the image is grayscale, apply histogram equalization directly
        contrast_image = cv2.equalizeHist(image)

    return contrast_image

In [ ]:
def increase_contrast_per_channel(image):
    """
    Increases the contrast of an image by applying histogram equalization to each color channel.

    Parameters:
    - image: Input image (in BGR format).

    Returns:
    - contrast_image: Contrast-enhanced image.
    """
    # Split the image into its BGR channels
    channels = cv2.split(image)

    # Apply histogram equalization to each channel
    equalized_channels = [cv2.equalizeHist(channel) for channel in channels]

    # Merge the channels back into a single image
    contrast_image = cv2.merge(equalized_channels)

    return contrast_image


In [ ]:
def final_frame_Y_contrast(image):
  compensated_image_exponential = compensate_color_exponential(image)
  compensated_image_rgb_exponential = cv2.cvtColor(compensated_image_exponential, cv2.COLOR_BGR2RGB)
  compensated_image_logarithmic = compensate_color_logarithmic(compensated_image_rgb_exponential)
  compensated_image_rgb_logarithmic = cv2.cvtColor(compensated_image_logarithmic, cv2.COLOR_BGR2RGB)
  contrast_image_after_log = increase_contrast(compensated_image_rgb_logarithmic)
  return contrast_image_after_log


In [ ]:
def final_frame_contrast_per_channel(image):
  contrast_image = increase_contrast_per_channel(image)
  return contrast_image

In [ ]:
import cv2

def process_video(input_video_path, output_video_path):
    # Open the input video
    cap = cv2.VideoCapture(input_video_path)

    # Check if the video was opened successfully
    if not cap.isOpened():
        print(f"Error: Cannot open video file {input_video_path}")
        return

    # Get video properties
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Initialize video writer
    codec = cv2.VideoWriter_fourcc(*'mp4v')  # Using mp4 codec
    out = cv2.VideoWriter(output_video_path, codec, fps, (width, height))

    # Check if the video writer was initialized successfully
    if not out.isOpened():
        print(f"Error: Cannot write to the output video file {output_video_path}")
        return

    # Process each frame
    frame_count = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Apply the final_frame function to each frame
        processed_frame = final_frame_contrast_per_channel(frame)

        # Write the processed frame to the output video
        out.write(processed_frame)

        frame_count += 1
        if frame_count % 10 == 0:  # Print progress every 10 frames
            print(f"Processed {frame_count} frames...")

    print(f"Processing completed. Total frames processed: {frame_count}")

    # Release resources
    cap.release()
    out.release()

# Path to input video and output video
input_video_path = '/content/new_pipe_video.mp4'  # Change to your input video path
output_video_path = '/content/output_video_new.mp4'

# Process the video
process_video(input_video_path, output_video_path)


Processed 10 frames...
Processed 20 frames...
Processed 30 frames...
Processed 40 frames...
Processed 50 frames...
Processed 60 frames...
Processed 70 frames...
Processed 80 frames...
Processed 90 frames...
Processed 100 frames...
Processed 110 frames...
Processed 120 frames...
Processed 130 frames...
Processed 140 frames...
Processed 150 frames...
Processed 160 frames...
Processed 170 frames...
Processed 180 frames...
Processed 190 frames...
Processed 200 frames...
Processed 210 frames...
Processed 220 frames...
Processed 230 frames...
Processed 240 frames...
Processing completed. Total frames processed: 248
